## Creating a Transaction Log Dataset

This notebook creates a Portfolio transaction log for the previous and current fiscal year (2023 - 2024 and 2024 - ). 

A fake transaction log using various stock tickers in US stock markets to build a portfolio dashboard is created. I used data from the Yahoo finance website (https://finance.yahoo.com/) for stock prices. The transaction log contains 500 transactions with the following column names - 
1) Stock Ticker
2) Transaction - Buy/Sell/Dividend
3) Date
4) Quantity - 0 for dividend transaction, negative numbers for sell transaction
5) Price - 0 for dividend transaction
6) Dividend - 0 for all except dividend transaction
7) Total Amount - () for Sell transactions, 0 for dividend transaction

The Number of Stocks Bought should be always greater than or equal to the number of stocks sold!

The dividend should be only from stocks that are in the Portfolio!

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
# Define the stock tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'JPM', 'V']

In [3]:
# Fetch stock price data for the dates defined as follows
start_date = '2023-01-01'
end_date = datetime.today()

In [4]:
# Download historical data for the tickers
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%%**********************]  10 of 10 completed


In [5]:
# Function to generate a random transaction log
def generate_transaction_log(tickers, data, num_transactions=500):
    transactions = []
    for _ in range(num_transactions):
        ticker = random.choice(tickers)
        transaction_type = random.choice(['Buy', 'Sell', 'Dividend'])
        date = data.index[random.randint(0, len(data) - 1)]
        price = data.loc[date, ticker]
        
        if transaction_type == 'Dividend':
            # Ensure dividend is only from stocks in the portfolio
            dividend_stocks = [t for t in tickers if random.random() < 0.5]  
            ticker = random.choice(dividend_stocks)
            dividend = round(random.uniform(0.5, 5.0), 2)
            transaction = [ticker, transaction_type, date.strftime('%Y-%m-%d'), 0, 0, dividend, 0]
        else:
            if transaction_type == 'Sell':
                # Ensure the number of stocks bought is greater than or equal to the number of stocks sold
                buy_quantity = random.randint(1, 100)
                sell_quantity = random.randint(1, buy_quantity)  # Adjusted to meet constraint
                quantity = -sell_quantity
            else:  # Buy transaction
                quantity = random.randint(1, 100)
            
            total_amount = round(quantity * price, 2)
            total_amount_str = f"({abs(total_amount)})" if transaction_type == 'Sell' else total_amount
            transaction = [ticker, transaction_type, date.strftime('%Y-%m-%d'), quantity, round(price, 2), 0, total_amount_str]
        
        transactions.append(transaction)
    
    return transactions

In [6]:
# Generate the transaction log
transactions = generate_transaction_log(tickers, data)

In [7]:
transactions_df = pd.DataFrame(transactions)
transactions_df

,0,1,2,3,4,5,6
0,NVDA,Buy,2023-10-24,29,43.65,0.00,1265.95
1,NFLX,Dividend,2024-01-26,0,0.00,1.80,0
2,NVDA,Dividend,2023-09-22,0,0.00,4.31,0
3,META,Sell,2023-07-07,-4,289.93,0.00,(1159.74)
4,GOOGL,Sell,2023-07-10,-17,116.32,0.00,(1977.38)
...,...,...,...,...,...,...,...
495,AAPL,Sell,2023-11-29,-65,188.87,0.00,(12276.71)
496,V,Sell,2023-10-09,-6,233.08,0.00,(1398.45)
497,AMZN,Buy,2023-07-25,34,129.13,0.00,4390.42
498,GOOGL,Buy,2023-05-31,24,122.73,0.00,2945.5


In [8]:
transactions_df = transactions_df.dropna()
transactions_df.isna().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

## Storing the Data into MySQL Database for better Data Management

In [9]:
#!pip install mysql-connector-python

In [10]:
import mysql.connector

In [11]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Godricshallow',
    database='transactions_log'
)
cursor = conn.cursor()

In [12]:
# Insert transaction log into MySQL
insert_query = """
INSERT INTO transactions_log (stock_ticker, transaction_type, date, quantity, price, dividend, total_amount)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [13]:
transactions = transactions_df.values.tolist()

In [14]:
cursor.executemany(insert_query, transactions)

# Commit and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully into the MySQL database!")

Data inserted successfully into the MySQL database!


## Meta Data

1. Stock Tickers: I selected 10 popular stock tickers for the portfolio.
2.	Fetch Stock Data: Using yfinance, I fetched the adjusted closing prices for these tickers within the given dates.
3.	Generate Transactions:
	- Randomly select a ticker and a transaction type (Buy, Sell, Dividend).
	- Pick a random date within the available data.
	- For buy/sell transactions, determine the quantity and calculate the total amount.
	-	For dividend transactions, randomly assign a dividend amount.
4.	Structure the Data: The data is stored in a DataFrame with columns for stock ticker, transaction type, date, quantity, price, dividend, and total amount.
5.	Save the Data: The transaction log is saved to an excel file named transaction_log.xlsx.

Additional Notes:

1) The quantity is positive for buy transactions and negative for sell transactions.
2) The price and total amount fields are void (empty) for dividend transactions.
3) The dividend field is only populated for dividend transactions.
4) The total amount for sell transactions is formatted with parentheses to indicate a negative value.